In [1]:
import pandas as pd
import numpy as np

### 级联操作
- pd.concat, pd.append

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
keys
join='outer' / 'inner':表示的是级联的方式，outer会将所有的项进行级联（忽略匹配和不匹配），而inner只会将匹配的项级联到一起，不匹配的不级联
ignore_index=False
```

- 匹配级联

In [2]:
df1 = pd.DataFrame(data=np.random.randint(0,100,size=(5,3)),columns=['A','B','C'])
df2 = pd.DataFrame(data=np.random.randint(0,100,size=(5,3)),columns=['A','D','C'])

In [3]:
pd.concat((df1,df1),axis=1) #行列索引都一致的级联叫做匹配级联

,A,B,C,A,B,C
0,30,64,14,30,64,14
1,9,56,65,9,56,65
2,33,16,25,33,16,25
3,57,26,91,57,26,91
4,0,84,64,0,84,64


- 不匹配级联
    - 不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致
    - 有2种连接方式：
        - 外连接：补NaN（默认模式）
        - 内连接：只连接匹配的项

In [4]:
pd.concat((df1,df2),axis=0)

,A,B,C,D
0,30,64.0,14,NaN
1,9,56.0,65,NaN
2,33,16.0,25,NaN
3,57,26.0,91,NaN
4,0,84.0,64,NaN
0,8,NaN,67,39.0
1,53,NaN,79,81.0
2,92,NaN,99,72.0
3,82,NaN,44,89.0
4,85,NaN,52,37.0


In [5]:
pd.concat((df1,df2),axis=0,join='inner') #inner直把可以级联的级联不能级联不处理

,A,C
0,30,14
1,9,65
2,33,25
3,57,91
4,0,64
0,8,67
1,53,79
2,92,99
3,82,44
4,85,52


- 如果想要保留数据的完整性必须使用outer（外连接）

- append函数的使用

In [6]:
df1.append(df1)

C:\Users\tigaly\AppData\Local\Temp\ipykernel_3676\2904621885.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df1)


,A,B,C
0,30,64,14
1,9,56,65
2,33,16,25
3,57,26,91
4,0,84,64
0,30,64,14
1,9,56,65
2,33,16,25
3,57,26,91
4,0,84,64


### 合并操作
- merge与concat的区别在于，merge需要依据某一共同列来进行合并

- 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

- 注意每一列元素的顺序不要求一致

#### 一对一合并

In [7]:
from pandas import DataFrame

In [8]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                'group':['Accounting','Engineering','Engineering'],
                })
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [9]:
df2 = DataFrame({'employee':['Lisa','Bob','Jake'],
                'hire_date':[2004,2008,2012],
                })
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


In [10]:
pd.merge(df1,df2,on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004


#### 一对多合并

In [11]:
df3 = DataFrame({
    'employee':['Lisa','Jake'],
    'group':['Accounting','Engineering'],
    'hire_date':[2004,2016]})
df3

,employee,group,hire_date
0,Lisa,Accounting,2004
1,Jake,Engineering,2016


In [12]:
df4 = DataFrame({'group':['Accounting','Engineering','Engineering'],
                       'supervisor':['Carly','Guido','Steve']
                })
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,Engineering,Steve


In [13]:
pd.merge(df3,df4)#on如果不写，默认情况下使用两表中公有的列作为合并条件

,employee,group,hire_date,supervisor
0,Lisa,Accounting,2004,Carly
1,Jake,Engineering,2016,Guido
2,Jake,Engineering,2016,Steve


#### 多对多合并

In [14]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                 'group':['Accounting','Engineering','Engineering']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [15]:
df5 = DataFrame({'group':['Engineering','Engineering','HR'],
                'supervisor':['Carly','Guido','Steve']
                })
df5

,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


In [16]:
pd.merge(df1,df5,how='right')

,employee,group,supervisor
0,Jake,Engineering,Carly
1,Lisa,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Guido
4,NaN,HR,Steve


In [17]:
pd.merge(df1,df5,how='left')

,employee,group,supervisor
0,Bob,Accounting,NaN
1,Jake,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Carly
4,Lisa,Engineering,Guido


#### key的规范化


- 当两张表没有可进行连接的列时，可使用left_on和right_on手动指定merge中左右两边的哪一列列作为连接的列

In [18]:
df1 = DataFrame({'employee':['Bobs','Linda','Bill'],
                'group':['Accounting','Product','Marketing'],
               'hire_date':[1998,2017,2018]})
df1

,employee,group,hire_date
0,Bobs,Accounting,1998
1,Linda,Product,2017
2,Bill,Marketing,2018


In [19]:
df5 = DataFrame({'name':['Lisa','Bobs','Bill'],
                'hire_dates':[1998,2016,2007]})
df5

,name,hire_dates
0,Lisa,1998
1,Bobs,2016
2,Bill,2007


In [20]:
pd.merge(df1,df5,left_on='employee',right_on='name')

,employee,group,hire_date,name,hire_dates
0,Bobs,Accounting,1998,Bobs,2016
1,Bill,Marketing,2018,Bill,2007


#### 内合并与外合并:out取并集 inner取交集

In [21]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [22]:
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [23]:
pd.merge(df6,df7,how='inner')

,name,food,drink
0,Mary,bread,wine


In [24]:
pd.merge(df6,df7,how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer
